### Kiva.org is a crowfunding platform to unlock capital for the underserved

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Loading the 3 main tables
loans = pd.read_csv('loans.csv')
lenders = pd.read_csv('lenders.csv')
loans_lenders = pd.read_csv('loans_lenders.csv')

In [3]:
#No duplicates
#loans[loans.duplicated()]
#lenders[lenders.duplicated()]
#loans_lenders[loans_lenders.duplicated()]

In [4]:
#No duolicates in the 2 columns that are used to join the 3 tables
#loans[loans.duplicated("loan_id")]
#lenders[lenders.duplicated(["permanent_name"])]
#loans_lenders[loans_lenders.duplicated(["loan_id"])]

In [5]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [6]:
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [7]:
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [8]:
loans.isnull().any()

loan_id                            False
loan_name                           True
original_language                   True
description                         True
description_translated              True
funded_amount                      False
loan_amount                        False
status                             False
activity_name                      False
sector_name                        False
loan_use                            True
country_code                        True
country_name                       False
town_name                           True
currency_policy                    False
currency_exchange_coverage_rate     True
currency                           False
partner_id                          True
posted_time                        False
planned_expiration_time             True
disburse_time                       True
raised_time                         True
lender_term                         True
num_lenders_total                  False
num_journal_entr

In [9]:
lenders.isnull().any()

permanent_name       False
display_name          True
city                  True
state                 True
country_code          True
member_since         False
occupation            True
loan_because          True
loan_purchase_num     True
invited_by            True
num_invited          False
dtype: bool

In [10]:
loans_lenders.isnull().any()

loan_id    False
lenders    False
dtype: bool

### There is no NULL in the columns used to join the 3 tables
### Some loans have more than 1 lenders because it's a crowdfunding, but we don't know for each lender what is the amount loaned, given a loan.

# 1) Normalize the loan_lenders table

In [11]:
# Check number of rows that we expected normalized operation
a = loans_lenders['lenders'].str.split(',').tolist()
count = 0
for i in a:
    count += len(i)
print(count)

28293931


In [12]:
loans_lenders = loans_lenders.set_index(['loan_id']).apply(lambda x: x.str.split(', ').explode()).reset_index()

In [13]:
print(loans_lenders.shape)

(28293931, 2)


In [14]:
loans_lenders.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


# 2) Add duration Column

In [15]:
loans[['loan_id','planned_expiration_time','disburse_time']].isnull().any()

loan_id                    False
planned_expiration_time     True
disburse_time               True
dtype: bool

In [16]:
#NULL in only one column
#loans['disburse_time'][loans['planned_expiration_time'].isnull()].unique()
#loans['planned_expiration_time'][loans['disburse_time'].isnull()].unique()

In [17]:
# Add Column duration to the loans table
loans['duration']=(pd.to_datetime(loans['planned_expiration_time']) - pd.to_datetime(loans['disburse_time']))/ np.timedelta64(1, 'D')


In [18]:
loans['duration'][loans['disburse_time'].isnull()].unique()

array([nan])

In [19]:
loans['duration'][loans['planned_expiration_time'].isnull()].unique()

array([nan])

In [20]:
loans[['loan_id','planned_expiration_time','disburse_time','duration']].head()

,loan_id,planned_expiration_time,disburse_time,duration
0,657307,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,53.812569
1,657259,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,96.600775
2,658010,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,37.548669
3,659347,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,34.798634
4,656933,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,57.923634


In [21]:
del(loans['duration'])

# 3) Lenders that have Funded at Least Twice 

In [22]:
lenders_count = loans_lenders.drop_duplicates()
lenders_count.shape

(27459086, 2)

#### Some lender has funded at least twice for the same loan.
#### Ther is no fund limit for a single loan so I don't want consider them as lenders that have funded at least twice

In [23]:
lenders_count = lenders_count.groupby(by='lenders', as_index=False).agg({'loan_id': pd.Series.nunique})

In [24]:
lenders_count['lenders'][lenders_count['loan_id']>1]

0                     000
1                   00000
2                    0002
4          0101craign0101
5                 0132575
                ...      
1383792           zzinnia
1383793        zzivnk7094
1383794          zzmcfate
1383796        zzrvmf8538
1383797            zzzsai
Name: lenders, Length: 893159, dtype: object

In [25]:
del(lenders_count)

# 4) For each country, compute how many loans have involved that country as borrowers

In [26]:
# loans['country_name'] doesn't have NULL value, loans['country_id'] does have it
loans.groupby('country_name')['loan_id'].count()

country_name
Afghanistan        2337
Albania            3075
Armenia           13952
Azerbaijan        10172
Belize              218
                  ...  
Vietnam           21839
Virgin Islands        2
Yemen              4206
Zambia             1277
Zimbabwe           5513
Name: loan_id, Length: 96, dtype: int64

# 5) For each country, compute the overall amount of money borrowed.

In [27]:
loans.groupby('country_name')['loan_amount'].sum()

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

# 6) Like the previous point, but expressed as a percentage of the overall amount lent

In [28]:
amount_tot = loans['loan_amount'].sum()

In [29]:
country_amount = loans.groupby(by='country_name', as_index=False).agg({'loan_amount': pd.Series.sum})
country_amount['loan_amount'] = country_amount['loan_amount']/amount_tot
country_amount

,country_name,loan_amount
0,Afghanistan,0.001666
1,Albania,0.003646
2,Armenia,0.019426
3,Azerbaijan,0.012514
4,Belize,0.000127
...,...,...
91,Vietnam,0.020891
92,Virgin Islands,0.000008
93,Yemen,0.002915
94,Zambia,0.001675


In [30]:
# Check that sum = 1
sum(country_amount['loan_amount'])

1.0

In [31]:
del(country_amount)

# 7) Like the three previous points, but split for each year (with respect to disburse time)

In [32]:
# Adding the column with the year from the 'disburse_time' column
loans['disburse_time_year'] =  pd.to_datetime(loans['disburse_time']).apply(lambda x: x.year)

In [33]:
# Point 4
loans.groupby(['country_name','disburse_time_year'])['loan_id'].count()

country_name  disburse_time_year
Afghanistan   2007.0                 408
              2008.0                 370
              2009.0                 678
              2010.0                 632
              2011.0                 247
                                    ... 
Zimbabwe      2013.0                 426
              2014.0                2078
              2015.0                 600
              2016.0                 808
              2017.0                1079
Name: loan_id, Length: 748, dtype: int64

In [34]:
# Point 5
loans.groupby(['country_name','disburse_time_year'])['loan_amount'].sum()

country_name  disburse_time_year
Afghanistan   2007.0                 194975.0
              2008.0                 365375.0
              2009.0                 585125.0
              2010.0                 563350.0
              2011.0                 245125.0
                                      ...    
Zimbabwe      2013.0                 678525.0
              2014.0                1311575.0
              2015.0                 723625.0
              2016.0                 788600.0
              2017.0                1237600.0
Name: loan_amount, Length: 748, dtype: float64

In [35]:
amount_sum = loans['loan_amount'].sum()

#values where disburse_time <> NULL
country_amount = loans.groupby(by=['country_name','disburse_time_year'], as_index=False).agg({'loan_amount': pd.Series.sum})
country_amount['loan_amount'] = country_amount['loan_amount']/amount_sum

#values where disburse_time = NULL
country_null = loans[loans['disburse_time'].isnull()].groupby(by=['country_name'], as_index=False).agg({'loan_amount': pd.Series.sum})
country_null['loan_amount'] = country_null['loan_amount']/amount_sum
country_null['disburse_time_year']=np.nan

#UNION
country_amount = pd.concat([country_amount,country_null], sort=True).sort_values(by=['country_name','disburse_time_year'])

country_amount

,country_name,disburse_time_year,loan_amount
0,Afghanistan,2007.0,0.000165
1,Afghanistan,2008.0,0.000309
2,Afghanistan,2009.0,0.000495
3,Afghanistan,2010.0,0.000477
4,Afghanistan,2011.0,0.000207
...,...,...,...
743,Zimbabwe,2013.0,0.000574
744,Zimbabwe,2014.0,0.001110
745,Zimbabwe,2015.0,0.000612
746,Zimbabwe,2016.0,0.000667


In [36]:
#Example to see that everything works fine
country_amount[country_amount['country_name'] == 'Kenya']

,country_name,disburse_time_year,loan_amount
297,Kenya,2005.0,0.000018
298,Kenya,2006.0,0.000189
299,Kenya,2007.0,0.001031
300,Kenya,2008.0,0.000357
301,Kenya,2009.0,0.000718
302,Kenya,2010.0,0.002818
303,Kenya,2011.0,0.004903
304,Kenya,2012.0,0.005731
305,Kenya,2013.0,0.006969
306,Kenya,2014.0,0.007890


In [37]:
sum(country_amount['loan_amount'])

0.9999999999999989

In [38]:
del(loans['disburse_time_year'])
del(amount_sum)
del(country_amount)

# 8) For each lender, compute the overall amount of money lent

#### loans_lenders has multiple loan_id, so if I merge the 2 tables I have duplicates in the amount column.
#### for the single loans I don't have any information regarding the distribution of the lenders so I consider a uniform distribution, that's mean that, given a loan, each lender has fuded (loan_amount)/(#lenders that lent for this loan) 

In [39]:
# 'lenders','loan_id','loan_amount' 
loans_lenders = loans_lenders.merge(loans[["loan_id",'loan_amount']],left_on='loan_id',right_on='loan_id',how='left')

In [40]:
loans_lenders.isnull().any()

loan_id        False
lenders        False
loan_amount     True
dtype: bool

In [41]:
loans[loans["loan_id"]==301]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model


In [42]:
loans_lenders[loans_lenders["loan_amount"].isnull()].shape

(19, 3)

#### there are 19 loans with Nan loan_amount

In [43]:
#Count how many lenders for each loans
loans_count = pd.DataFrame(loans_lenders.groupby(['loan_id']).count()).drop(columns=['loan_amount']).rename(columns={"lenders": "count"})
loans_count.head()


,count
loan_id,
84,3
85,2
86,3
88,3
89,4


In [44]:
loans_count["count"].sum()

28293931

In [45]:
loans_lenders = loans_lenders.merge(loans_count,left_on='loan_id',right_on='loan_id',how='left')
loans_lenders["real_amount"] = loans_lenders["loan_amount"] /loans_lenders["count"] 
del(loans_lenders["loan_amount"])
del(loans_lenders["count"])

loans_lenders

,loan_id,lenders,real_amount
0,483693,muc888,30.625
1,483693,sam4326,30.625
2,483693,camaran3922,30.625
3,483693,lachheb1865,30.625
4,483693,rebecca3499,30.625
...,...,...,...
28293926,1206425,trogdorfamily7622,40.625
28293927,1206425,danny6470,40.625
28293928,1206425,don6118,40.625
28293929,1206486,alan5175,75.000


In [46]:
#loans_lenders["real_amount"].sum()

In [47]:
#lenders_amount["real_amount"].sum()

In [48]:
#loans["loan_amount"].sum()

In [49]:
lenders_amount = pd.DataFrame(loans_lenders.groupby(["lenders"])["real_amount"].sum())
lenders_amount.head()

,real_amount
lenders,
000,1764.285078
00000,1380.693644
0002,2472.563566
00mike00,52.631579
0101craign0101,2623.565117


In [50]:
# Difference % between the sum of all loans and the loans breaked by lenders
(loans["loan_amount"].sum() - lenders_amount["real_amount"].sum())/loans["loan_amount"].sum()

0.015238155253774163

In [51]:
(loans["loan_amount"].sum() - lenders_amount["real_amount"].sum())/28293931

0.6362822118990734

DA RIVEDERE

1.5% of difference seems high but it's possible if we consider that we had 28.293.931 divisions and we lost something in each approssimation

In [52]:
del(lenders_amount)
del(loans_count)
del(loans_lenders["real_amount"])

# 9) For each country, difference between the overall amount of money lent and the overall amount of money borrowed. 
## Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

##### I'll work with the country_id because it' more sure then the country_name for the merge between the country from the 2 tables

In [53]:
loans["country_name"][loans["country_code"].isnull()].unique()

array(['Namibia'], dtype=object)

In [54]:
# mapping loans["country_code"]="NA" where loans["country_name"] = "Namibia"
loans["country_code"] = loans.apply(lambda x: "NA" if x["country_name"]=="Namibia" else x["country_code"], axis=1)

In [55]:
# loan_amount borrowed per country
loans_country = loans.groupby(by='country_code', as_index=False).agg({'loan_amount': pd.Series.sum})
loans_country

,country_code,loan_amount
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0
...,...,...
91,XK,3083025.0
92,YE,3444000.0
93,ZA,1006525.0
94,ZM,1978975.0


In [56]:
# loan_amount lent per country
lenders_country = loans_lenders.merge(loans[["loan_id","loan_amount"]],left_on='loan_id',right_on='loan_id',how='left') [["lenders","loan_amount"]]
lenders_country = lenders_country.merge(lenders[["permanent_name","country_code"]],left_on='lenders',right_on='permanent_name',how='left')[["country_code","loan_amount"]]

# Values used to increase the loan_amount assuming that the true distribution among the countries is the same
tot_shape = lenders_country.shape[0]
notNull_shape = lenders_country[lenders_country["country_code"].notnull()].shape[0]

# Aggregation and avoid the Nan
lenders_country = lenders_country.groupby(by='country_code', as_index=False).agg({'loan_amount': pd.Series.sum})
lenders_country = lenders_country[lenders_country["country_code"].notnull()]

# Increase the loan_amount according without modify the distribution
lenders_country["loan_amount"] = lenders_country["loan_amount"] * tot_shape / notNull_shape

lenders_country

,country_code,loan_amount
0,AD,2.744085e+05
1,AE,9.752719e+07
2,AF,6.943458e+06
3,AG,2.921132e+04
4,AI,7.431704e+04
...,...,...
220,YE,8.904237e+05
221,YT,4.019625e+03
222,ZA,3.435626e+07
223,ZM,2.136139e+06


In [57]:
# Merge the 2 tables and calculate the difference
country_difference = lenders_country.merge(loans_country,left_on='country_code',right_on='country_code',how='outer',suffixes = ("_lenders","_loans"))
# fillna(0) because if the record for the country is missing means that no loan has funded in that country
country_difference["difference_amount"] = country_difference["loan_amount_lenders"].fillna(0) - country_difference["loan_amount_loans"].fillna(0)

del(country_difference["loan_amount_lenders"])
del(country_difference["loan_amount_loans"])

country_difference

,country_code,difference_amount
0,AD,2.744085e+05
1,AE,9.752719e+07
2,AF,4.975508e+06
3,AG,2.921132e+04
4,AI,7.431704e+04
...,...,...
224,ZW,-5.450956e+06
225,GZ,-5.000000e+03
226,MR,-1.500000e+04
227,NA,-3.952500e+04


In [58]:
del(lenders_country)
del(loans_country)

# 10) Which country has the highest ratio between the difference computed at the previous point and the population?

In [59]:
# load the table with the information about the population and filter it 
country_stats = pd.read_csv('country_stats.csv')

In [60]:
#country_stats = country_stats[["country_name","country_code","population"]]

In [61]:
# keep the country_name to know the merge which country_code will miss 
country_ratio = country_difference.merge(country_stats[["country_name","country_code","population"]],left_on='country_code',right_on='country_code',how='left')

In [62]:
country_ratio["country_code"][country_ratio["population"].isnull()].unique()

array(['AD', 'AG', 'AI', 'AN', 'AQ', 'AS', 'AW', 'BB', 'BM', 'BS', 'BV',
       'CK', 'CS', 'CV', 'CX', 'DM', 'EH', 'FK', 'FO', 'GD', 'GF', 'GI',
       'GL', 'GP', 'GS', 'GU', 'IO', 'KN', 'KY', 'LC', 'LI', 'MC', 'MH',
       'MO', 'MP', 'MQ', 'NC', 'NF', 'PF', 'PN', 'PW', 'RE', 'SC', 'SJ',
       'SM', 'TC', 'TK', 'TO', 'TV', 'TW', 'UM', 'VA', 'VC', 'VG', 'VI',
       'VU', 'WF', 'YT', 'GZ', 'NA'], dtype=object)

In [63]:
country_ratio[country_ratio["population"].isnull()].shape[0]

60

#### There are 60 out of 229 countries without the population value

In [64]:
country_ratio = country_ratio[country_ratio["population"].notnull()].merge(loans[["country_code","country_name"]].drop_duplicates(),left_on='country_code',right_on='country_code',how='left',suffixes = ("_ratio","_loans"))
country_ratio 

,country_code,difference_amount,country_name_ratio,population,country_name_loans
0,AE,9.752719e+07,United Arab Emirates,9400145.0,NaN
1,AF,4.975508e+06,Afghanistan,35530081.0,Afghanistan
2,AL,-3.636686e+06,Albania,2930187.0,Albania
3,AM,-2.152621e+07,Armenia,2930450.0,Armenia
4,AO,2.155931e+06,Angola,29784193.0,NaN
...,...,...,...,...,...
164,ZA,3.334973e+07,South Africa,56717156.0,South Africa
165,ZM,1.571643e+05,Zambia,17094130.0,Zambia
166,ZW,-5.450956e+06,Zimbabwe,16529904.0,Zimbabwe
167,MR,-1.500000e+04,Mauritania,4420184.0,Mauritania


In [65]:
#country_name = country_ratio["country_name_loans"][country_ratio["country_name_ratio"].isnull()][country_ratio["country_name_loans"].notnull()]
#country_name 

In [66]:
#Guam
#country_stats["country_name"][country_stats["country_name"].str.contains("ua", regex=False)]

In [67]:
#Saint Vincent and the Grenadines
#country_stats["country_name"][country_stats["country_name"].str.contains("inc|rena", regex=True)]

In [68]:
#Virgin Islands
#country_stats["country_name"][country_stats["country_name"].str.contains("rgi|land", regex=True)]

In [69]:
#Vanuatu
#country_stats["country_name"][country_stats["country_name"].str.contains("anua", regex=False)]

In [70]:
#Gaza
#country_stats["country_name"][country_stats["country_name"].str.contains("gaza", case=False, regex=False)]

In [71]:
#Nimibia
#country_stats["country_name"][country_stats["country_name"].str.contains("Nimibia", case=False, regex=False)]

In [72]:
#elimino i null

#country_ratio = country_difference.merge(country_stats,left_on='country_code',right_on='country_code',how='left')
country_ratio["ratio_amount"] = country_ratio["difference_amount"]/country_ratio["population"]
#country_ratio = country_ratio[country_ratio["population"].notnull()]
del(country_ratio["difference_amount"])
del(country_ratio["population"])
del(country_ratio["country_name_ratio"])
del(country_ratio["country_name_loans"])

country_ratio

,country_code,ratio_amount
0,AE,10.375072
1,AF,0.140036
2,AL,-1.241111
3,AM,-7.345701
4,AO,0.072385
...,...,...
164,ZA,0.588001
165,ZM,0.009194
166,ZW,-0.329763
167,MR,-0.003394


# 11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [73]:
#country_stats = pd.read_csv('country_stats.csv')
#country_stats = country_stats[["country_name","country_code","population_below_poverty_line"]]

country_ratio = country_difference.merge(country_stats[["country_name","country_code","population_below_poverty_line"]],left_on='country_code',right_on='country_code',how='left')

country_ratio[country_ratio["population_below_poverty_line"].isnull()].shape[0]

81

In [74]:
country_ratio = country_ratio[country_ratio["population_below_poverty_line"].notnull()]

In [75]:
#country_ratio = country_ratio.merge(loans[["country_code","country_name"]].drop_duplicates(),left_on='country_code',right_on='country_code',how='left',suffixes = ("_ratio","_loans"))
#country_name = country_ratio["country_name_loans"][country_ratio["country_name_ratio"].isnull()][country_ratio["country_name_loans"].notnull()]

#country_name

In [76]:

country_ratio = country_difference.merge(country_stats,left_on='country_code',right_on='country_code',how='left')
country_ratio["ratio_amount"] = country_ratio["difference_amount"]/country_ratio["population_below_poverty_line"]


country_ratio["country_code"][country_ratio["ratio_amount"].idxmax()]

'US'

# 12) For each year, compute the total amount of loans. 
### Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. 
#### For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [77]:
loans_by_year = loans[["loan_id","disburse_time","planned_expiration_time","loan_amount"]]
#loans_by_year = loans_by_year.merge(loans_lenders,left_on="loan_id", right_on="loan_id", how = "right")
loans_by_year

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22 08:00:00.000 +0000,2014-02-14 03:30:06.000 +0000,125.0
1,657259,2013-12-20 08:00:00.000 +0000,2014-03-26 22:25:07.000 +0000,400.0
2,658010,2014-01-09 08:00:00.000 +0000,2014-02-15 21:10:05.000 +0000,400.0
3,659347,2014-01-17 08:00:00.000 +0000,2014-02-21 03:10:02.000 +0000,625.0
4,656933,2013-12-17 08:00:00.000 +0000,2014-02-13 06:10:02.000 +0000,425.0
...,...,...,...,...
1419602,988180,2015-11-23 08:00:00.000 +0000,2016-01-02 01:00:03.000 +0000,400.0
1419603,988213,2015-11-24 08:00:00.000 +0000,2016-01-02 16:40:07.000 +0000,300.0
1419604,989109,2015-11-13 08:00:00.000 +0000,2016-01-03 22:20:04.000 +0000,2425.0
1419605,989143,2015-11-03 08:00:00.000 +0000,2016-01-05 08:50:02.000 +0000,100.0


In [78]:
loans_by_year.isnull().any()

loan_id                    False
disburse_time               True
planned_expiration_time     True
loan_amount                False
dtype: bool

In [79]:
loans_by_year[loans_by_year["planned_expiration_time"].isnull()].shape

(371834, 4)

In [80]:
loans_by_year[loans_by_year["disburse_time"].isnull()].shape

(2813, 4)

In [81]:
loans_by_year[loans_by_year["disburse_time"]==loans_by_year["planned_expiration_time"]].shape

(0, 4)

In [82]:
loans_by_year[loans_by_year["disburse_time"]>loans_by_year["planned_expiration_time"]].shape

(14935, 4)

In [83]:
loans_by_year = loans_by_year[loans_by_year["planned_expiration_time"].notnull()]
loans_by_year = loans_by_year[loans_by_year["disburse_time"].notnull()]
loans_by_year = loans_by_year[loans_by_year["disburse_time"]<=loans_by_year["planned_expiration_time"]]

loans_by_year.head()


,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22 08:00:00.000 +0000,2014-02-14 03:30:06.000 +0000,125.0
1,657259,2013-12-20 08:00:00.000 +0000,2014-03-26 22:25:07.000 +0000,400.0
2,658010,2014-01-09 08:00:00.000 +0000,2014-02-15 21:10:05.000 +0000,400.0
3,659347,2014-01-17 08:00:00.000 +0000,2014-02-21 03:10:02.000 +0000,625.0
4,656933,2013-12-17 08:00:00.000 +0000,2014-02-13 06:10:02.000 +0000,425.0


In [84]:
(loans.shape[0] - loans_by_year.shape[0])/loans.shape[0]

0.27442806354152943

In [85]:
loans_by_year["disburse_time"] = pd.to_datetime(loans_by_year["disburse_time"]).dt.date
loans_by_year["planned_expiration_time"] = pd.to_datetime(loans_by_year["planned_expiration_time"]).dt.date

In [86]:
loans_by_year.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22,2014-02-14,125.0
1,657259,2013-12-20,2014-03-26,400.0
2,658010,2014-01-09,2014-02-15,400.0
3,659347,2014-01-17,2014-02-21,625.0
4,656933,2013-12-17,2014-02-13,425.0


In [87]:
loans_by_year.apply(lambda x: loans_by_year["disburse_time"].apply(lambda x: x.year) if 
                    loans_by_year["disburse_time"].apply(lambda x: x.year).sub(loans_by_year["planned_expiration_time"].apply(lambda x: x.year) > 0).bool() else -1, axis=1)

ValueError: ('The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().', 'occurred at index 0')

In [ ]:
loans_by_year["planned_expiration_time"] - loans_by_year["disburse_time"]

In [ ]:
loans_by_year["planned_expiration_time"].apply(lambda x: x.year)

In [ ]:
loans_by_year.apply(lambda x: "NA" if x["country_name"]=="Namibia" else x["country_code"], axis=1)